# Get top options with good `sdMult` and `rom`

In [1]:
MARKET = 'NSE'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID
PAPER = ibp.PAPER

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_ohlcs.pkl', 'df_opts.pkl', 'df_symlots.pkl',
       'df_und_margins.pkl', 'df_unds.pkl', 'qopts.pkl'], dtype='<U18')

In [11]:
df = df_opts.sort_values('rom', ascending = False).drop(['contract'], 1)
m = (df.rom > ibp.MINEXPROM) & (df.sdMult > 1.8)
r = (df.strike > df.undPrice) & (df.right == 'C') | (df.strike < df.undPrice) & (df.right == 'P')
df1 = df[m & r].sort_values(['right', 'strike'], ascending=[True, False])

In [13]:
df1.symbol.unique()

array(['BOSCHLTD', 'HEROMOTOC', 'BAJAJ-AUT', 'TCS', 'COFORGE',
       'ASIANPAIN', 'HINDUNILV', 'RELIANCE', 'INDIGO', 'ACC', 'PIDILITIN',
       'BATAINDIA', 'PVR', 'MUTHOOTFI', 'LT', 'INFY', 'UBL', 'GODREJPRO',
       'SRTRANSFI', 'RAMCOCEM', 'INDUSINDB', 'GRASIM', 'TECHM', 'HAVELLS',
       'HCLTECH', 'VOLTAS', 'AXISBANK', 'GODREJCP', 'MCDOWELL-',
       'ICICIBANK', 'DABUR', 'TATACONSU', 'CUMMINSIN', 'TVSMOTOR',
       'BHARTIART', 'IGL', 'UPL', 'BPCL', 'ADANIPORT', 'MARICO',
       'TATACHEM', 'WIPRO', 'JSWSTEEL', 'TORNTPOWE', 'AMBUJACEM', 'SBIN',
       'ZEEL', 'DLF', 'POWERGRID', 'ITC', 'EXIDEIND', 'MMFIN',
       'TATAMOTOR', 'COALINDIA', 'VEDL', 'RECLTD', 'PFC', 'BEL', 'NMDC',
       'NTPC', 'IOC', 'LTFH', 'TATAPOWER', 'FEDERALBN', 'BHEL',
       'NATIONALU', 'GMRINFRA', 'IDEA', 'MRF', 'BAJAJFINS', 'BAJFINANC',
       'SIEMENS', 'PETRONET', 'IBULHSGFI'], dtype=object)

In [15]:
df1[df1.symbol == 'HEROMOTOC']

,conId,symbol,strike,expiry,secType,right,lot,dte,und_iv,undPrice,comm,margin,bid,ask,close,last,iv,price,sdMult,qty,intrinsic,timevalue,rom
4715,442413799,HEROMOTOC,3300.0,20201126,OPT,C,300.0,5,0.382835,2988.5,20.0,169858.32,-1.0,-1.0,9.90,NaN,0.382835,9.90,2.326241,300.0,0.0,9.90,1.267821
3448,442413790,HEROMOTOC,3250.0,20201126,OPT,C,300.0,5,0.382835,2988.5,20.0,187119.57,-1.0,-1.0,12.65,NaN,0.382835,12.65,1.952848,300.0,0.0,12.65,1.472721
6246,442413707,HEROMOTOC,2700.0,20201126,OPT,P,300.0,5,0.382835,2988.5,20.0,170165.81,-1.0,-1.0,8.50,NaN,0.382835,8.50,2.154480,300.0,0.0,8.50,1.085353


In [16]:
from support import get_prob
get_prob(1.8)

0.9281393617741486